In [94]:
import numpy as np 
import pandas as pd 
import scipy as sp 
import matplotlib.pyplot as plt
import seaborn
import random
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import zipfile


from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelBinarizer, StandardScaler, normalize
from sklearn.manifold import TSNE

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from keras.constraints import non_neg, unit_norm, max_norm




%matplotlib inline
plt.rcParams['figure.figsize'] = 15,  8

In [98]:
import datetime

print(datetime.datetime.now())

2017-07-19 16:15:14.161634


In [11]:
val_data = pd.read_csv('data/numerai_tournament_data.csv')
valIndx = ~val_data.target.isnull()
data = pd.read_csv('data/numerai_training_data.csv')

(96,)

In [12]:
def load_data():
    zip_ref = zipfile.ZipFile('data/numerai_datasets.zip', 'r')
    zip_ref.extractall('data/')
    zip_ref.close()
    data = pd.read_csv('data/numerai_training_data.csv')
    val_data = pd.read_csv('data/numerai_tournament_data.csv')
    X_val = val_data.iloc[:, 3:-1]
    
    features = data.iloc[:, 3:-1]
    labels = data.iloc[:, -1]
    
    
    return features.values, labels.values, data['era'].values, X_val.values, val_data.id.values


def create_model():
    model = Sequential()
    model.add(Dense(600, input_dim=21,
             kernel_regularizer=l2(0.00005 ),
             activity_regularizer=l1(0.000005),
             kernel_constraint=max_norm()))
    model.add(Activation('tanh'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(256,
             kernel_regularizer=l2(0.00005 ),
             activity_regularizer=l1(0.000005),
             kernel_constraint=max_norm()))
    model.add(Activation('tanh'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(256,
             kernel_regularizer=l2(0.00005 ),
             activity_regularizer=l1(0.000005),
             kernel_constraint=max_norm()))
    model.add(Activation('tanh'))
    # model.add(Dropout(rate=0.5))
    model.add(Dense(1, activation='sigmoid'))


    sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    return model
    
    
    
def train_and_evaluate__model(model, features, labels, trainIndx, testIndx):
    model.fit(features[trainIndx], labels[trainIndx],
              validation_data=(features[testIndx], labels[testIndx]), epochs=5, batch_size=128, verbose=0)
    

In [13]:
n_folds = 2
features, labels, era, x_vals, val_ids = load_data()
skf = StratifiedKFold(n_splits=n_folds, shuffle=True)
skf.get_n_splits(features, era)
n_models = 5

predictions = []
for j in range(n_models):
    for i, (trainIndx, testIndx) in enumerate(skf.split(features, era)):
        print("model:", j, ", Running Fold, ", i+1, "/", n_folds)
        model = None # Clearing the NN.
        model = create_model()
        train_and_evaluate__model(model, features, labels, trainIndx, testIndx)
        predictions.append(model.predict_proba(x_vals, verbose=0))

model: 0 , Running Fold,  1 / 96
model: 0 , Running Fold,  2 / 96
model: 0 , Running Fold,  3 / 96
model: 0 , Running Fold,  4 / 96
model: 0 , Running Fold,  5 / 96
model: 0 , Running Fold,  6 / 96
model: 0 , Running Fold,  7 / 96
model: 0 , Running Fold,  8 / 96


KeyboardInterrupt: 

In [1]:
predictions

NameError: name 'predictions' is not defined

In [16]:
preds = np.asarray(predictions)
preds = preds.reshape(preds.shape[0], preds.shape[1])

## averaging predictions

In [10]:
probs = preds.mean(axis=0)

# wierd prod way

In [14]:
probs = np.power(np.prod(preds, axis=0), 1.0 / preds.shape[0])

# Weighted by NN

In [81]:
def nn_weight_model(n_models, pred_probs, labels):
    model = Sequential()
    model.add(Dense(1, input_dim=n_models, kernel_constraint=unit_norm()))
    model.add(Dense(1, activation='sigmoid'))


    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(pred_probs, labels, epochs=20, batch_size=128, verbose=0)
    weights = model.get_weights()
    return weights[0].flatten()

w = nn_weight_model(7, preds[:, valIndx].T, val_data.target.values[valIndx])

In [18]:
submit = pd.DataFrame({'id':val_ids, 'probability': probs.flatten()})
submit.to_csv('submit.csv', index=False)

In [63]:
preds.T[0]

array([ 0.50370014,  0.50790846,  0.50588727,  0.5082491 ,  0.50093037,
        0.50217307,  0.48731211], dtype=float32)

In [93]:
np.sqrt(np.power(w, 2).sum())

0.99999988

In [66]:
(np.multiply(w, preds.T[0])).mean()

0.14806812

In [1]:
S = input()

5 ab bb hefg dhck dkhc


In [11]:
for s in S.split():
    if len(s) < 2: continue
    
    s_ = list(s)
    i = len(s)
    while s >= s:
        
        
    
    print(s)
    
    

ab
bb
hefg
dhck
dkhc


In [61]:
s_ = list(S.split()[3])

i = len(s_)
s_.insert(i - 2, s_.pop(i - 1))
i -= 1

s_

['h', 'e', 'g', 'f']

In [80]:
s_ =  list(S.split()[3])

a = s_.pop(-1)
indx = list(map(lambda x: x < a, s_))

In [83]:
indx.index(True)

1